In [1]:
import os 
import re
import pickle
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from function import *

import sentencepiece as spm
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence

In [2]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer

In [3]:
path = os.getcwd()

In [4]:
# spm_load
sp = spm.SentencePieceProcessor()
sp.Load(path + '/out/cate_spm.model')
# tkn_load
with open(path + '/out/cate_tkn.pickle', 'rb') as handle:
    tkn = pickle.load(handle)
# label load
with open(path + '/out/labels.pickle', 'rb') as handle:
    mapping_dct = pickle.load(handle)
# model_load
classification_model = keras.models.load_model(path + '/out/cate_model.h5')

In [5]:
class_names = list(mapping_dct.values())
explainer = LimeTextExplainer(class_names = class_names)

In [6]:
def pred(lst, max_len = 50):
    pre = [' '.join(clean_spm(sp.encode_as_pieces(text))) for text in lst]
    t = sequence.pad_sequences(tkn.texts_to_sequences(pre), maxlen = max_len)
    P = classification_model.predict_on_batch(t)
    return P

In [7]:
def spm(text):
    r = [' '.join(clean_spm(sp.encode_as_pieces(text))) for text in [text]]
    return ''.join(r)

In [8]:
def cate_pred_two(lst, max_len):
    pre = [' '.join(clean_spm(sp.encode_as_pieces(text))) for text in lst]
    t = sequence.pad_sequences(tkn.texts_to_sequences(pre), maxlen = max_len)
    P = classification_model.predict_on_batch(t)
    pred = np.argsort(P)[0][::-1][:2]
    prob = np.partition(P.flatten(), -2)[::-1][:2]
    X = pd.Series(pred).map(mapping_dct).to_list()
    return dict(zip(X, prob))

In [9]:
import random
import pymongo

client = pymongo.MongoClient("mongodb://133.186.168.8:50000")
db = client.AP_ECOMMERCE
collection = db.EC_PROCESS_LOG
cursor = collection.find({'collect_site':'top.ssg.com'})
dt = pd.DataFrame(list(cursor))
dt['text'] = dt['goods_name'] + ' ' + dt['option_name'] + ' ' + dt['master_brand_name']
dt.shape

(219844, 23)

In [10]:
text = random.choice(dt['text'].to_list())
pre = spm(text)
exp = explainer.explain_instance(pre, pred, num_features = 10, top_labels = 2)
# print(exp.available_labels())
print ('goods_name:',dt[dt['text'] == text]['goods_name'].values[0])
print ('option_name:',dt[dt['text'] == text]['option_name'].values[0])
print('traverse_cate:',dt[dt['text'] == text]['traverse_cate'].values[0])
print('goods_cate:',dt[dt['text'] == text]['goods_cate'].values[0])
print ('url:',dt[dt['text'] == text]['collect_url'].values[0])
print ('=' * 130)
P = cate_pred_two([text],50)
print ('preds:',P)
print ('=' * 130)
print ('Weight For Words\n')
reverse_mapping_dct = {v: k for k, v in mapping_dct.items()}
print ('\n'.join(map(str, exp.as_list(label = reverse_mapping_dct[list(P.keys())[0]]))))
print ('=' * 130,'\n')
exp.show_in_notebook(text = True)

goods_name: [SIXPAD]식스패드 트레이닝 슈트 암&웨스트세트 Arm & Waist set
option_name: M_S
traverse_cate: ssg > 뷰티 > 미용기기/소품 > 페이스관리기기 > 페이셜관리기
goods_cate: ssg > 뷰티 > 미용기기/소품 > 페이스관리기기 > 페이셜관리기
url: http://www.ssg.com/item/itemView.ssg?itemId=1000024908979&siteNo=6004&salestrNo=6005
preds: {'메이크업세트': 0.60186946, '립앤아이리무버': 0.19332413}
Weight For Words



NameError: name 'reverse_mapping_dct' is not defined

In [16]:
reverse_mapping_dct = {v: k for k, v in mapping_dct.items()}

In [17]:
reverse_mapping_dct

{'BB/CC크림': 0,
 '구강청결제': 1,
 '남성메이크업 (남성)': 2,
 '남성향수': 3,
 '네일리무버': 4,
 '네일컬러': 5,
 '네일케어용품': 6,
 '네일팁/스티커': 7,
 '데오드란트': 8,
 '드라이샴푸': 9,
 '로션': 10,
 '로션 (남성)': 11,
 '린스/컨디셔너': 12,
 '립라이너/립팔레트': 13,
 '립밤/립케어': 14,
 '립스틱': 15,
 '립앤아이리무버': 16,
 '마스카라': 17,
 '메이크업베이스': 18,
 '메이크업세트': 19,
 '미스트': 20,
 '바디로션/크림': 21,
 '바디미스트/코롱': 22,
 '바디선케어/선크림': 23,
 '바디스크럽/필링': 24,
 '바디오일': 25,
 '바디워시': 26,
 '바디파우더': 27,
 '베이비로션': 28,
 '베이비워시/클렌져': 29,
 '뷰티용품': 30,
 '블러셔': 31,
 '비누': 32,
 '샴푸': 33,
 '선스틱': 34,
 '선스프레이': 35,
 '선케어/BB크림 (남성)': 36,
 '선쿠션/파우더': 37,
 '스킨/애프터쉐이브 (남성)': 38,
 '스킨/토너': 39,
 '슬리핑팩': 40,
 '시트마스크': 41,
 '아이메이크업': 42,
 '아이크림': 43,
 '에센스 (남성)': 44,
 '에센스/앰플': 45,
 '여성향수': 46,
 '염색약': 47,
 '워시오프팩': 48,
 '입욕제/버블바스': 49,
 '제모기/제모스트럽/제모크림': 50,
 '청결제': 51,
 '컨실러': 52,
 '컬링에센스': 53,
 '코팩': 54,
 '쿠션파운데이션': 55,
 '크림': 56,
 '크림 (남성)': 57,
 '클렌징/쉐이빙 (남성)': 58,
 '클렌징오일': 59,
 '클렌징워터': 60,
 '클렌징크림/로션': 61,
 '클렌징티슈': 62,
 '클렌징폼/젤': 63,
 '틴트/립글로스': 64,
 '파마약': 65,
 '파우더/팩트': 66,
 '파운데이션': 67,
 '페